In [53]:
# import tensorflow as tf
import numpy as np
import h2o
import os
import pandas as pd

# from .feature_engineering_filter import Find_correct_port

In [54]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [55]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [56]:
train_data_preprocessed = train_data
train_data_preprocessed.loc[train_data_preprocessed["cog"] >= 360, "cog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["sog"] >= 1023, "sog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["rot"] == -128, "rot"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["heading"] == 511, "heading"] = (
    np.nan
)


pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)


train_data_preprocessed = train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())


train_data_preprocessed = (
    train_data_preprocessed.groupby("vesselId")
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
)


print(train_data_preprocessed.head())

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)

train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)


# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01" + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    + "-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9] + "59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "23:59",
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["etaRaw"] - train_data_preprocessed["time"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689  
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19  
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126  
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3  


/var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/ipykernel_29320/776026068.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.ffill().bfill())


                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-12 14:07:47  308.1  17.1 -6.0    316.0        0  01-08 06:00   
1 2024-01-12 14:31:00  307.6  17.3  5.0    313.0        0  01-14 23:30   
2 2024-01-12 14:57:23  306.8  16.9  5.0    312.0        0  01-14 23:30   
3 2024-01-12 15:18:48  307.9  16.9  6.0    313.0        0  01-14 23:30   
4 2024-01-12 15:39:47  307.0  16.3  7.0    313.0        0  01-14 23:30   

   latitude  longitude                  vesselId                    portId  
0   7.50361   77.58340  61e9f38eb937134a3c4bfd8b  61d376b393c6feb83e5eb50c  
1   7.57302   77.49505  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  
2   7.65043   77.39404  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  
3   7.71275   77.31394  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  
4   7.77191   77.23585  61e9f38eb937134a3c4bfd8b  61d376d893c6feb83e5eb546  


In [57]:
train_latitude_radians = np.deg2rad(train_data_preprocessed["latitude"])
train_longitude_radians = np.deg2rad(train_data_preprocessed["longitude"])
train_cog_radians = np.deg2rad(train_data_preprocessed["cog"])
train_heading_radians = np.deg2rad(train_data_preprocessed["heading"])
train_hour = np.deg2rad(train_data_preprocessed["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data_preprocessed["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data_preprocessed["time"].dt.month * 360 / 12)

print(train_data_preprocessed["latitude"].head(10))
print(train_latitude_radians.head(10))


train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)

train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_preprocessed["latitude_sin"] = train_latitude_sin
train_data_preprocessed["latitude_cos"] = train_latitude_cos
train_data_preprocessed["longitude_sin"] = train_longitude_sin
train_data_preprocessed["longitude_cos"] = train_longitude_cos
train_data_preprocessed["cog_sin"] = train_cog_sin
train_data_preprocessed["cog_cos"] = train_cog_cos
train_data_preprocessed["heading_sin"] = train_heading_sin
train_data_preprocessed["heading_cos"] = train_heading_cos

train_data_preprocessed["hour_sin"] = train_hour_sin
train_data_preprocessed["hour_cos"] = train_hour_cos
train_data_preprocessed["day_sin"] = train_day_sin
train_data_preprocessed["day_cos"] = train_day_cos
train_data_preprocessed["month_sin"] = train_month_sin
train_data_preprocessed["month_cos"] = train_month_cos


train_data_preprocessed = train_data_preprocessed.drop(
    columns=["latitude", "longitude", "cog", "heading", "portId"], axis=1
)
print(train_data_preprocessed.head(10))

0    7.50361
1    7.57302
2    7.65043
3    7.71275
4    7.77191
5    7.81285
6    7.86929
7    7.92585
8    7.98258
9    8.03598
Name: latitude, dtype: float64
0    0.130963
1    0.132174
2    0.133525
3    0.134613
4    0.135645
5    0.136360
6    0.137345
7    0.138332
8    0.139322
9    0.140254
Name: latitude, dtype: float64
                 time   sog  rot  navstat                  vesselId  \
0 2024-01-12 14:07:47  17.1 -6.0        0  61e9f38eb937134a3c4bfd8b   
1 2024-01-12 14:31:00  17.3  5.0        0  61e9f38eb937134a3c4bfd8b   
2 2024-01-12 14:57:23  16.9  5.0        0  61e9f38eb937134a3c4bfd8b   
3 2024-01-12 15:18:48  16.9  6.0        0  61e9f38eb937134a3c4bfd8b   
4 2024-01-12 15:39:47  16.3  7.0        0  61e9f38eb937134a3c4bfd8b   
5 2024-01-12 15:54:48  16.1  5.0        0  61e9f38eb937134a3c4bfd8b   
6 2024-01-12 16:14:59  16.1 -6.0        0  61e9f38eb937134a3c4bfd8b   
7 2024-01-12 16:35:24  16.0  2.0        0  61e9f38eb937134a3c4bfd8b   
8 2024-01-12 16:55:24  16.0 -

In [58]:
def Last_known_location_training_data(
    data: pd.DataFrame, shift_lenghts
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data

    """
    all_test_data = pd.DataFrame()

    for shift_length in shift_lenghts:

        grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))

        grouped_data["time_diff"] = (
            grouped_data["time"].diff(-shift_length).dt.total_seconds().abs()
        )

        original_time_and_id = grouped_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        shifted_data = grouped_data.shift(shift_length)
        shifted_data[
            [
                "last_latitude_sin",
                "last_latitude_cos",
                "last_longitude_sin",
                "last_longitude_cos",
            ]
        ] = shifted_data[
            ["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]
        ]

        shifted_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ] = original_time_and_id[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        # Drops all values with nan values
        result = shifted_data.dropna().reset_index(drop=True)

        all_test_data = pd.concat([all_test_data, result], ignore_index=True)

    # Uncomment the line below if you want to remove the "time" column after processing
    # data = data.drop("time", axis=1)

    return all_test_data

In [59]:
print(train_data_preprocessed.columns)

train_data_shifted_df = Last_known_location_training_data(
    train_data_preprocessed, [10,25]
)

# train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'sog', 'rot', 'navstat', 'vesselId', 'seconds_to_eta',
       'latitude_sin', 'latitude_cos', 'longitude_sin', 'longitude_cos',
       'cog_sin', 'cog_cos', 'heading_sin', 'heading_cos', 'hour_sin',
       'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos'],
      dtype='object')


/var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/ipykernel_29320/3212926780.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))
/var/folders/3t/hy3nmqqx6f70nkbvw0n8lbh80000gn/T/ipykernel_29320/3212926780.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("ti

In [60]:
def append_last_known_data_test(
    test_data: pd.DataFrame, known_data: pd.DataFrame
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
    """

    if not test_data["vesselId"].isin(known_data["vesselId"]).all():
        missing_vessels = test_data[
            ~test_data["vesselId"].isin(known_data["vesselId"])
        ]["vesselId"].unique()
        raise ValueError(
            f"The following vesselIds are missing in known_data: {missing_vessels}"
        )
    print(
        test_data[~test_data["vesselId"].isin(known_data["vesselId"])][
            "vesselId"
        ].unique()
    )

    grouped_data = (
        known_data.sort_values("time")
        .groupby("vesselId")
        .tail(1)
        .reset_index(drop=True)
    )
    original_time = test_data[["time"]]
    test_data = test_data.drop("time", axis=1)

    result = pd.merge(test_data, grouped_data, how="left", on="vesselId")

    result["time_diff"] = (original_time["time"] - result["time"]).dt.total_seconds()

    return result

In [61]:
test_data_with_last_known_df = append_last_known_data_test(
    test_data, train_data_preprocessed
)
test_data_with_last_known_df[
    [
        "last_latitude_sin",
        "last_latitude_cos",
        "last_longitude_sin",
        "last_longitude_cos",
    ]
] = test_data_with_last_known_df[
    [
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ]
]
test_data_with_last_known_df = test_data_with_last_known_df.drop(
    columns=[
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ],
    axis=1,
)

print(test_data_with_last_known_df.head())

[]
   ID                  vesselId  scaling_factor                time   sog  \
0   0  61e9f3aeb937134a3c4bfe3d             0.3 2024-05-07 23:48:16   0.0   
1   1  61e9f473b937134a3c4c02df             0.3 2024-05-07 23:57:16   0.0   
2   2  61e9f469b937134a3c4c029b             0.3 2024-05-07 23:59:08  18.7   
3   3  61e9f45bb937134a3c4c0221             0.3 2024-05-07 23:52:34   0.1   
4   4  61e9f38eb937134a3c4bfd8d             0.3 2024-05-07 23:51:29   0.3   

   rot  navstat  seconds_to_eta   cog_sin   cog_cos  ...  hour_cos   day_sin  \
0  0.0        5       -133396.0  0.006981 -0.999976  ...  0.965926  0.994522   
1  0.0        5       -521836.0  0.417867  0.908508  ...  0.965926  0.994522   
2  0.0        0         45952.0  0.139173  0.990268  ...  0.965926  0.994522   
3  0.0        1        -81454.0 -0.625243  0.780430  ...  0.965926  0.994522   
4  0.0        2        101311.0 -0.933580  0.358368  ...  0.965926  0.994522   

    day_cos  month_sin  month_cos  time_diff  last_la

In [62]:
h2o.init(max_mem_size="7g")

train_data_shifted = h2o.H2OFrame(train_data_shifted_df)
test_data_with_last_known = h2o.H2OFrame(test_data_with_last_known_df)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 26 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_kristofferseyffarth_2brlyz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.949 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [63]:
train_data_shifted_without_validation, validation_data_shifted = (
    train_data_shifted.split_frame(ratios=[0.9], seed=42)   
)               

In [64]:
features_lat = [
    # "vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    "rot",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
]
features_long = [
    # "vesselId",
    "cog_sin",
    "cog_cos",
    "sog",
    "rot",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
    "latitude_sin",
    "latitude_cos",
]
target_long_sin = "longitude_sin"
target_long_cos = "longitude_cos"
target_lat_sin = "latitude_sin"
target_lat_cos = "latitude_cos"

In [65]:
params = {
    "ntrees": 200,  # Maximum number of trees
    "max_depth": 10,  # Maximum depth of each tree
    "min_rows": 15,  # Minimum number of rows per leaf
    "learn_rate": 0.1,  # Learning rate
    "sample_rate": 0.8,  # Row sample rate per tree
    "col_sample_rate": 0.8,  # Column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.2,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.001,  # Tolerance for early stopping
}
# score_eval_metric_only if you want to predict only on the evaluation dataset, could help against overfitting

gbm_lat_sin = h2o.estimators.H2OXGBoostEstimator(**params)
gbm_lat_cos = h2o.estimators.H2OXGBoostEstimator(**params)
gbm_long_sin = h2o.estimators.H2OXGBoostEstimator(**params)
gbm_long_cos = h2o.estimators.H2OXGBoostEstimator(**params)

# gbm_cog = h2o.estimators.H2OXGBoostEstimator()
# gbm_sog = h2o.estimators.H2OXGBoostEstimator()
# gbm_rot = h2o.estimators.H2OXGBoostEstimator()
# gbm_heading = h2o.estimators.H2OXGBoostEstimator()
# gbm_navstat = h2o.estimators.H2OXGBoostEstimator()
# # gbm_etaRaw = h2o.esti#mators.H2OXGBoostEstimator() #Remove etaRaw because it requires preprocessing
# # gbm_portId = h2o.estimators.H2OXGBoostEstimator()


In [66]:
gbm_lat_sin.train(
    x=features_lat,
    y=target_lat_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |

/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729616899047_9


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.0006145554280860213
RMSE: 0.024790228479907587
MAE: 0.0073725731543322565
RMSLE: 0.026706982536021762
Mean Residual Deviance: 0.0006145554280860213

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.0008113264005314822
RMSE: 0.02848379189173173
MAE: 0.00790900499013612
RMSLE: 0.030744633495960568
Mean Residual Deviance: 0.0008113264005314822

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance      validation_rmse       validation_mae        validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ---------------------  --------------------  --------------------  ---------------------
     2024-10-22 20:39:38  0.087 sec          0.0                0.35953032397409873   0.27348894912264937    0.12926205385692038    0.3611453293538869    0.27432289067186216   0.13042594891412743
     2024-10-22 20:39:58  20.106 sec         1.0                0.3245911305037683    0.24681388635766022    0.10535940200171436    0.3260161459911256    0.24755200760405616   0.10628652744690693
     2024-10-22 20:40:29  51.612 sec         3.0                0.26520217621245357   0.20137473897467306    0.07033219426782127    0.2663466648649903    0.2019769034124578    0.07094054588470346
     2024-10-22 20:41:01  1 min 22.973 sec   5.0                0.21745738267655773   0.16468393305316104    0.047287713280538865   0.21836435418717023   0.16518251147837504   0.04768299117957993
     2024-10-22 20:41:16  1 min 37.793 sec   6.0                0.19715919809441831   0.14902011792364653    0.03887174939323408    0.19794864974035228   0.14946702770824125   0.039183667934028665
     2024-10-22 20:41:30  1 min 51.895 sec   7.0                0.17896806912760735   0.1349241444121326     0.03202956976726404    0.17969256212899437   0.13533721462851153   0.0322894168844825
     2024-10-22 20:41:56  2 min 17.805 sec   9.0                0.1483267984666657    0.11089323990886842    0.02200083914337086    0.14893271068684905   0.11124408101363194   0.022180952312532677
     2024-10-22 20:42:21  2 min 43.425 sec   11.0               0.12390397134268813   0.09141639586140358    0.015352194114489681   0.12441683439104285   0.09171423299174206   0.015479548679888185
     2024-10-22 20:42:45  3 min  6.919 sec   13.0               0.10458254527666262   0.07563105948103699    0.010937508776545186   0.10507552167606903   0.07589561418211449   0.01104086525549805
     2024-10-22 20:43:05  3 min 26.985 sec   15.0               0.08951197782226589   0.06288800308600995    0.00801239417365382    0.09001500830173047   0.06312644740420889   0.008102701719560606
---  ---                  ---                ---                ---                   ---                    ---                    ---                   ---                   ---
     2024-10-22 20:50:05  10 min 27.396 sec  60.0               0.035590975580733604  0.00971256432739456    0.0012667175427883755  0.03756728190429474   0.009991484406718699  0.0014113006696767508
     2024-10-22 20:51:13  11 min 35.512 sec  69.0               0.033973277875576975  0.009290364686454592   0.0011541836096111682  0.036216756817427156  0.009601020326104439  0.0013116534743726563
     2024-10-22 20:52:27  12 min 49.039 sec  79.0               0.032790527434508804  0.009026634014279828   0.0010752186894332743  0.035214762934431855  0.009365232605762958  0.0012400795285282358
     2024-10-22 20:53:51  14 min 12.964 sec  90.0               0.03149373700129936   0.008759212543198358   0.0009918554703070123  0.03415835246820094   0.009131269104595911  0.0011667930433418494
     2024-10-22 20:55:23  15 min 45.66

In [67]:
gbm_lat_cos.train(
    x=features_lat,  # .append(latitude_sin)
    y=target_lat_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729616899047_10


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 8.779297004476659e-05
RMSE: 0.009369790288195706
MAE: 0.00381836949059959
RMSLE: 0.005192559965383918
Mean Residual Deviance: 8.779297004476659e-05

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.00010202472037600978
RMSE: 0.010100728705197947
MAE: 0.00400470082622367
RMSLE: 0.00558671910901927
Mean Residual Deviance: 0.00010202472037600978

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance       validation_rmse       validation_mae         validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ----------------------  --------------------  ---------------------  ----------------------
     2024-10-22 21:11:20  0.090 sec          0.0                0.26580951030414834   0.23339721520724913    0.07065469576813113     0.26583576702495715   0.2334561551947388     0.0706686550297473
     2024-10-22 21:11:39  19.047 sec         1.0                0.23940519526140472   0.21008222351630648    0.057314847518151316    0.23943482871066687   0.21014193267613251    0.05732903719970638
     2024-10-22 21:12:08  47.949 sec         4.0                0.17507021132586878   0.15323400771438603    0.030649578893684352    0.17511186772275272   0.1532968860088273     0.030664166217350845
     2024-10-22 21:12:26  1 min  5.959 sec   6.0                0.14226288155267275   0.12420843526598       0.020238727467669797    0.14230817393820044   0.12427028699357609    0.020251616369625115
     2024-10-22 21:12:45  1 min 25.344 sec   8.0                0.11574163173671675   0.10073715177517058    0.013396125317077758    0.11579059584301726   0.10079693440672734    0.013407462085680966
     2024-10-22 21:13:14  1 min 53.624 sec   11.0               0.08524773546931959   0.07368332680570103    0.0072671764026470895   0.08530178932217065   0.07374522954975764    0.007276395261563986
     2024-10-22 21:13:44  2 min 23.771 sec   14.0               0.06321686099507326   0.054048545873545274   0.003996371514070414    0.06327364187348071   0.05410599660587021    0.004003553755933491
     2024-10-22 21:14:12  2 min 51.536 sec   17.0               0.04744732938499662   0.039895164891658806   0.002251249065768364    0.0475083905917308    0.039947580627835634   0.0022570471766164557
     2024-10-22 21:14:32  3 min 11.765 sec   19.0               0.0395431578441092    0.03274175552351368    0.0015636613322841351   0.03961724951975686   0.032795296326512104   0.0015695264595106755
     2024-10-22 21:15:02  3 min 42.379 sec   22.0               0.030715968415378977  0.02458598477473816    0.0009434707156945589   0.03080392640748775   0.024640756768495087   0.0009488818821179212
---  ---                  ---                ---                ---                   ---                    ---                     ---                   ---                    ---
     2024-10-22 21:22:16  10 min 55.734 sec  64.0               0.012500653341903202  0.004817288223843042   0.0001562663339744357   0.012797338549652178  0.004912952012524802   0.00016377187395441372
     2024-10-22 21:23:42  12 min 21.568 sec  73.0               0.012118571565981118  0.004663328804599698   0.00014685977679980604  0.012447137468346896  0.0047664484403434956  0.00015493123115592518
     2024-10-22 21:25:30  14 min  9.483 sec  84.0               0.011687926611884805  0.004537568953335074   0.000136607628484805    0.012060814080693996  0.004650323263704401   0.00014546323628906657
     2024-10-22 21:27:52  16 min 32.142 sec  98.0               0.011360698595000916  0.0044391898863618585  0.0001290654725664558   0.011774761598735499  0.004561353038692376   0.000138645010

In [68]:
performance_lat_sin = gbm_lat_sin.model_performance(test_data=validation_data_shifted)
performance_lat_cos = gbm_lat_cos.model_performance(test_data=validation_data_shifted)


# Print the performance metrics
print(performance_lat_sin)
print(performance_lat_cos)

ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.0008113264005314822
RMSE: 0.02848379189173173
MAE: 0.00790900499013612
RMSLE: 0.030744633495960568
Mean Residual Deviance: 0.0008113264005314822
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.00010202472037600978
RMSE: 0.010100728705197947
MAE: 0.00400470082622367
RMSLE: 0.00558671910901927
Mean Residual Deviance: 0.00010202472037600978


In [69]:
gbm_long_sin.train(
    x=features_long,
    y=target_long_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729616899047_11


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.00024298775060437096
RMSE: 0.015588064363620358
MAE: 0.005633010991734151
RMSLE: NaN
Mean Residual Deviance: 0.00024298775060437096

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.0003719669741866556
RMSE: 0.019286445348654986
MAE: 0.006019494184953743
RMSLE: NaN
Mean Residual Deviance: 0.0003719669741866556

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance       validation_rmse       validation_mae         validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ----------------------  --------------------  ---------------------  ----------------------
     2024-10-22 21:41:12  0.054 sec          0.0                0.6973162521011392    0.5570587987953262     0.48624995544437954     0.697220866820864     0.5570385456848005     0.4861169371304371
     2024-10-22 21:41:29  16.680 sec         1.0                0.6282538416854593    0.5018440421509343     0.3947028895925382      0.6281973747435289    0.5018531003677379     0.3946319416346616
     2024-10-22 21:41:49  36.453 sec         3.0                0.5100204250417733    0.40727336269782294    0.260120833959791       0.5100207761755353    0.40732101048156055    0.26012119213069546
     2024-10-22 21:42:07  54.625 sec         5.0                0.4144220488376878    0.3306982188472786     0.1717456345628269      0.414483504578507     0.3307732240452761     0.17179657556768124
     2024-10-22 21:42:25  1 min 12.818 sec   7.0                0.3370723901597744    0.2686677019039503     0.1136177962080232      0.3371961617696755    0.26877532354421185    0.11370125151220116
     2024-10-22 21:42:45  1 min 32.623 sec   9.0                0.27446153468159573   0.21838138891753692    0.07532913401977678     0.2746799510916063    0.21851580410478416    0.07544907553168723
     2024-10-22 21:43:03  1 min 50.636 sec   11.0               0.2239486666763486    0.1777033458949623     0.05015300530611429     0.22425982475970327   0.17786071764309408    0.050292469001252824
     2024-10-22 21:43:21  2 min  8.376 sec   13.0               0.18323598573412508   0.14480741294872815    0.03357542646795649     0.1836316388667454    0.14497090543096164    0.0337205787928868
     2024-10-22 21:43:39  2 min 26.766 sec   15.0               0.15054564831862238   0.11823687145232968    0.022663992227674328    0.15106982819878412   0.11841779918837748    0.022822092992010146
     2024-10-22 21:44:05  2 min 53.157 sec   18.0               0.11317196076875986   0.08759964077835324    0.012807892704245722    0.11385352256331455   0.08777991750694825    0.012962624600075176
---  ---                  ---                ---                ---                   ---                    ---                     ---                   ---                    ---
     2024-10-22 21:54:00  12 min 47.232 sec  73.0               0.0224167425679433    0.007530568291882173   0.0005025103473574412   0.025532117215942833  0.007829222328037268   0.0006518890095286445
     2024-10-22 21:55:40  14 min 27.881 sec  82.0               0.02163978736101699   0.007253140566887076   0.00046828039703003066  0.02483425925761334   0.007563958993049823   0.0006167404328743539
     2024-10-22 21:57:36  16 min 23.495 sec  92.0               0.02080133175601328   0.007010414933762454   0.0004326954028237265   0.02405204078147741   0.00733218086025946    0.0005785006657538526
     2024-10-22 21:59:49  18 min 36.319 sec  103.0              0.01989003687399747   0.006818312880165628   0.00039561356684897917  0.023166362287404336  0.00714336998314384    0.0005366803416312699
     2024-10-22 22:02:13  21 min  0.

In [70]:
gbm_long_cos.train(
    x=features_long,  # .append("longitude_sin")
    y=target_long_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |

██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729616899047_12


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.0001444365766068645
RMSE: 0.012018176925260525
MAE: 0.00320643295102143
RMSLE: NaN
Mean Residual Deviance: 0.0001444365766068645

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.00022768324166031313
RMSE: 0.015089176308212226
MAE: 0.0034708732544838175
RMSLE: NaN
Mean Residual Deviance: 0.00022768324166031313

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance       validation_rmse       validation_mae         validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ----------------------  --------------------  ---------------------  ----------------------
     2024-10-22 22:18:49  0.020 sec          0.0                0.6766950419793908    0.6008440122333621     0.45791617983948946     0.6781214557626759    0.6021292653116523     0.45984870876569084
     2024-10-22 22:19:08  18.666 sec         1.0                0.6095322598566565    0.5411182850381947     0.3715295758059626      0.6108131366507961    0.5422715633877424     0.3730926879051841
     2024-10-22 22:19:53  1 min  4.577 sec   5.0                0.401753645272861     0.3561984142034901     0.16140599149003182     0.40260563956073053   0.356947529200613      0.1620913010061049
     2024-10-22 22:20:17  1 min 28.286 sec   7.0                0.3264241819380892    0.2891105946453518     0.10655274655395075     0.32713591260593766   0.2897101146403867     0.10701790531651967
     2024-10-22 22:20:39  1 min 50.211 sec   9.0                0.2655069841645177    0.2347660057446954     0.07049395864013745     0.26612161868547535   0.235252678114128      0.07082071593177755
     2024-10-22 22:21:02  2 min 13.244 sec   11.0               0.2161656346188478    0.19067531963112228    0.04672758159016921     0.21670983892934165   0.19107676193036993    0.0469631542887812
     2024-10-22 22:21:27  2 min 38.622 sec   13.0               0.17633505551060907   0.15500247385084373    0.031094051801929586    0.17684145290949146   0.15532640233246858    0.03127289946713989
     2024-10-22 22:22:01  3 min 12.364 sec   16.0               0.1305686130933535    0.11374609876838923    0.01704816272512184     0.13104497060663955   0.11398812171136287    0.017172784321295022
     2024-10-22 22:22:35  3 min 45.789 sec   19.0               0.09737028571730777   0.0836160042225164     0.009480972540670148    0.09790084112511563   0.08380990093888412    0.009584574693005133
     2024-10-22 22:23:08  4 min 19.474 sec   22.0               0.0736375766532612    0.061666424890320176   0.00542249269536492     0.07426882653259721   0.06182866090784449    0.005515858594529015
---  ---                  ---                ---                ---                   ---                    ---                     ---                   ---                    ---
     2024-10-22 22:33:30  14 min 41.424 sec  76.0               0.01668510045649776   0.0042316101955428625  0.00027839257724342176  0.019113414775619092  0.004422966137188568   0.00036532262438485427
     2024-10-22 22:35:06  16 min 17.493 sec  85.0               0.016136257460613077  0.004089714931857588   0.0002603788048351912   0.018619505067745543  0.004286486622653696   0.00034668596896780194
     2024-10-22 22:36:59  18 min 10.008 sec  95.0               0.015569071871578076  0.003966806292092058   0.00024239599894236366  0.01810854689635471   0.00417309233347362    0.0003279194706974777
     2024-10-22 22:38:57  20 min  8.515 sec  106.0              0.015028268730225685  0.0038647354317280338  0.0002258488610278791   0.017598288954568785  0.004076652640787674   0.0003096997741284977
     2024-10-22 22:41:23  22 min 3

In [71]:
performance_long_sin = gbm_long_sin.model_performance(test_data=validation_data_shifted)
performance_long_cos = gbm_long_cos.model_performance(test_data=validation_data_shifted)

print(performance_long_sin)
print(performance_long_cos)

ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.0003719669741866556
RMSE: 0.019286445348654986
MAE: 0.006019494184953743
RMSLE: NaN
Mean Residual Deviance: 0.0003719669741866556
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.00022768324166031313
RMSE: 0.015089176308212226
MAE: 0.0034708732544838175
RMSLE: NaN
Mean Residual Deviance: 0.00022768324166031313


In [72]:
test_data_with_predicted_lat = test_data_with_last_known

lat_predictions_sin = gbm_lat_sin.predict(test_data_with_last_known)
test_data_with_last_known["latitude_sin"] = lat_predictions_sin
lat_predictions_cos = gbm_lat_cos.predict(test_data_with_last_known)
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

test_data_with_predicted_lat["latitude_sin"] = lat_predictions_sin
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

long_predictions_sin = gbm_long_sin.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_sin"] = long_predictions_sin
long_predictions_cos = gbm_long_cos.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_cos"] = long_predictions_cos

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [73]:
# Convert sine and cosine values back to radians
lat_predictions_sin = lat_predictions_sin.as_data_frame()
lat_predictions_cos = lat_predictions_cos.as_data_frame()
long_predictions_sin = long_predictions_sin.as_data_frame()
long_predictions_cos = long_predictions_cos.as_data_frame()


lat_predictions_radians = np.arctan2(lat_predictions_sin, lat_predictions_cos)
long_predictions_radians = np.arctan2(long_predictions_sin, long_predictions_cos)

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)

# Print the first few rows to verify the conversion
print(lat_predictions_degrees.head())
print(long_predictions_degrees.head())

/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.venv/lib/python3.12/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/Users/kristofferseyffarth/Downloads/lokalFiles/emner/Maskinlering/Gruppe/.v

     predict
0  31.295841
1  15.531776
2  38.851931
3 -43.442703
4  48.517376
      predict
0  -80.615617
1  119.876571
2   11.227556
3  172.985014
4   -5.972330


In [74]:
def create_prediction_visualization_data(validation_data):
    lat_val_sin = gbm_lat_sin.predict(validation_data)
    lat_val_cos = gbm_lat_cos.predict(validation_data)
    long_val_sin = gbm_long_sin.predict(validation_data)
    long_val_cos = gbm_long_cos.predict(validation_data)

    lat_val_sin = lat_val_sin.as_data_frame()
    lat_val_cos = lat_val_cos.as_data_frame()
    long_val_sin = long_val_sin.as_data_frame()
    long_val_cos = long_val_cos.as_data_frame()

    validation_data = validation_data.as_data_frame()

    lat_val_radians = np.arctan2(lat_val_sin, lat_val_cos)
    long_val_radians = np.arctan2(long_val_sin, long_val_cos)

    evaluation_lat_radians = np.arctan2(
        validation_data["latitude_sin"], validation_data["latitude_cos"]
    )
    evaluation_long_radians = np.arctan2(
        validation_data["longitude_sin"], validation_data["longitude_cos"]
    )

    # Convert radians to degrees
    lat_val_degrees = np.rad2deg(lat_val_radians)
    long_val_degrees = np.rad2deg(long_val_radians)

    evaluation_lat_degrees = np.rad2deg(evaluation_lat_radians)
    evaluation_long_degrees = np.rad2deg(evaluation_long_radians)

    eval_predictions = pd.concat([lat_val_degrees, long_val_degrees], axis=1)

    eval_actual = pd.concat([evaluation_lat_degrees, evaluation_long_degrees], axis=1)

    eval_predictions.columns = ["latitude_predicted", "longitude_predicted"]
    eval_actual.columns = ["latitude", "longitude"]

    eval = pd.DataFrame()
    eval[["latitude_predicted", "longitude_predicted"]] = eval_predictions
    eval[["latitude", "longitude"]] = eval_actual
    eval[["vesselId", "time"]] = validation_data[["vesselId", "time"]]
    eval.to_csv("eval_predictions.csv")


# create_prediction_visualization_data(validation_data_shifted)

In [75]:
predictions = pd.concat([lat_predictions_degrees, long_predictions_degrees], axis=1)
predictions.columns = ["latitude_predicted", "longitude_predicted"]

In [76]:
predictions["ID"] = test_data["ID"]
predictions = predictions[["ID", "longitude_predicted", "latitude_predicted"]]

In [77]:
print(predictions.columns)

Index(['ID', 'longitude_predicted', 'latitude_predicted'], dtype='object')


In [78]:
predictions.to_csv("predictions.csv", index=False)